# This notebook is created to organize data from MetaCyc into one dataframe

### What kind of data is available by MetaCyc zipped file?
Noted that we recieved data from Pathway Tools platform version 22.6

**Important data that we extracted**
- compounds.dat
- enzrxns.dat
- reactions.dat

**Somewhat useful**
- atom-mapping and atom-mappint-smiles
- pathways

**May not be useful**
- classes
- gene_association
- genes
- metabolic-reactions.xml
- proteins
- protligandcplxes
- protseq
- pubs
- regulation
- rnas
- species
- transporters

In [47]:
import numpy as np
import pandas as pd
import math
import ast

### This is how our desired master dataframe look from last quarter

For now, make a dataset with enzyme/compounds linked to each other and then we can use RDKit-based function and existing functions to generate Mol-files, Distance, and Negative data

master_df = pd.read_csv('../datasets/MASTER_DF.csv')

master_df.head()

feature_df = master_df[['PubChem', 'dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
        'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU']]
feature_df.set_index(keys=['PubChem'], inplace=True)
feature_df.head()

## The current database from MetaCyc starts here

In [48]:
df_cpd = pd.read_csv('df_cpd.csv', index_col = 0)
df_rxn = pd.read_csv('parsed_rxns.csv', index_col = 0)
df_enz = pd.read_csv('df_enzrxns.csv', index_col = 0)
df_cpd = df_cpd.set_index(keys ='UNIQUE-ID')
df_rxn = df_rxn.set_index(keys = 'UNIQUE-ID')
df_enz = df_enz.set_index(keys = 'UNIQUE-ID')

# Here is an important note
The data from csv reading is in text formatted which lead to some problematic data handling.
- Some int data was read as float e.g. PubChemID which can be fixed by .astype() shown in cell #4
- List was formatted in text which required decoding by ast.literal_eval(x) which was written as a fucntion below

In [49]:
def recover_list(df, column):
    """This function will recover a list formatted string read from .csv into a list"""
    assert type(df[column][0]) != type([]), "TypeError: The data type is already a list, it should not be converted again"
    replacement = []
    for index, row in df.iterrows():
        
        data = []
        
        if type(row[column]) == type('string'):
            data = ast.literal_eval(row[column])
        else:
            pass
        replacement.append(data)
    df[column] = replacement
    return

In [50]:
# Change PubChemID into int type in df_cpd
PubChemID_int = df_cpd['PubChemID'].fillna(0).astype(int)
df_cpd['PubChemID'] = PubChemID_int

# Recover list format of df_rxn
rxn_list_fix = ['EC-NUMBER', 'ERXN-NUMBER', 'SUBSTRATES', 'PRODUCTS']
for col in rxn_list_fix:
    recover_list(df_rxn, col)

# Recover list format of df_enz
enz_list_fix = ['REACTION', 'ALTERNATIVE-SUBSTRATES', '^SUBSTRATE', 'KM', 'KCAT', 'VMAX']
for col in enz_list_fix:
    recover_list(df_enz, col)

##### the code below verify that all `.astype()` of PubChemID data didn't change the data unless it is `nan`
df_cpd = pd.read_csv('df_cpd.csv', index_col = 0)
df_cpd = df_cpd.set_index(keys ='UNIQUE-ID')
PubChemID_int = df_cpd['PubChemID'].fillna(0).astype(int)
for i in range(PubChemID_int.shape[0]):
    
#print (df_cpd['PubChemID'].loc['CPD-14743'])
#a = df_cpd['PubChemID'].loc['CPD-14743']
    if PubChemID_int[i] != df_cpd['PubChemID'][i]:
        print(PubChemID_int[i], df_cpd['PubChemID'][i])

In [11]:
df_cpd.head()

,COMMON-NAME,GIBBS-0,INCHI,SMILES,PubChemID
UNIQUE-ID,,,,,
CPD-14966,"(2Z,4Z)-2-hydroxy-5-carboxymuconate-6-semialde...",-150.733760,InChI=1S/C7H6O6/c8-3-4(6(10)11)1-2-5(9)7(12)13...,[CH](=O)C(C(=O)[O-])=CC=C(O)C([O-])=O,90657979
CPD-14905,"4&alpha;,14&alpha;-dimethyl-porifersta-8,25(27...",568.928300,InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-...,CCC(C(C)=C)CCC(C)[CH]3(CCC4(C)(C2(CC[CH]1(C(C)...,102515047
CPD-14658,(+)-orobanchyl acetate,90.736885,InChI=1S/C21H24O7/c1-10-8-14(27-19(10)23)25-9-...,CC1(C(=O)OC(C=1)OC=C2(C(=O)O[CH]3([CH]2C(OC(C)...,24796587
CPD-14885,3-aminobenzoate,12.324662,InChI=1S/C7H7NO2/c8-6-3-1-2-5(4-6)7(9)10/h1-4H...,C(=O)([O-])C1(C=C(N)C=CC=1),3014145
CPD-14884,3-amino-4-hydroxybenzenesulfonate,-40.905334,"InChI=1S/C6H7NO4S/c7-5-3-4(12(9,10)11)1-2-6(5)...",C1(C(S(=O)([O-])=O)=CC(N)=C(O)C=1),4146016


In [12]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [13]:
df_enz.head()

,COMMON-NAME,ENZYME,REACTION,ALTERNATIVE-SUBSTRATES,^SUBSTRATE,KM,KCAT,VMAX,PH-OPT,SPECIFIC-ACTIVITY,TEMPERATURE-OPT
UNIQUE-ID,,,,,,,,,,,
ENZRXN-15384,maleylacetate reductase,MONOMER-14207,[MALEYLACETATE-REDUCTASE-RXN],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-26017,neurosporene &beta;-cyclase,MONOMER-20355,[RXN-8038],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-18089,&beta;-<i>N</i>-acetylhexosaminidase,CPLX-8130,[3.2.1.52-RXN],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-19053,cellulase,MONOMER-16501,[RXN-2043],"[(CELLULOSE |Lichenin|), (CELLULOSE |Xylogluca...",[],[],[],[],NaN,NaN,NaN
ENZRXN-952,IPP isomerase,YPL117C-MONOMER,[IPPISOM-RXN],[],[],[],[],[],NaN,NaN,NaN


`&beta;-<i>N</i>-acetylhexosaminidase` refer to __&beta;-<i>N</i>-acetylhexosaminidase__

Some weird notation here
- RXN-12314 doesn't have any enzyme affilated
- However, it links to a certain enzymatic reaction ENZRXN-19061 which has the reaction linked back as well
- What I don't understand is ENZYME column from enzrxn dataset which link to somewhere else that we cannot find an enzyme database in this current version of MetaCyc database
    - This ENZYME name and common name match with ENZYME name in rxn dataset. However, this is still unclear

##### Write a function that call a `UNIQUE-ID` or `COMMON-NAME` into InChI
This is an easiest approach to link df_rxn with df_cpd

In [51]:
def get_inchi(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    
    inchi = df_cpd['INCHI'][ID]
    
    return inchi

def get_smiles(ID):
    
    """This function accept UNIQUE-ID and return SMILES string of a certain compound"""
    
    smiles = df_cpd['SMILES'][ID]
    
    return smiles

In [52]:
def get_pubchem(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    if ID in df_cpd['PubChemID']:
        pubchem = df_cpd['PubChemID'][ID]
    else:
        pubchem = '0'
        
    return pubchem

In [195]:
get_inchi('CPD-7557')

'InChI=1S/C15H22O/c1-10-4-6-13-11(2)5-7-14(12(3)9-16)15(13)8-10/h8-9,11,13-15H,3-7H2,1-2H3/t11-,13+,14+,15+/m1/s1'

In [197]:
get_smiles('CPD-7557')

'CC1(CC[CH]2(C(CC[CH]([CH](C=1)2)C(C=O)=C)C))'

df_enz.loc['ENZRXN-19061']

type(int(df_cpd['PubChemID'].loc['CPD-14966']))

df_cpd.loc['5-AMINO-LEVULINATE']

df_cpd.loc['PROTON']

df_rxn.head()

df_rxn.loc['3.2.1.52-RXN']

To strip the string see method below
>>> import ast
>>> x = u'[ "A","B","C" , " D"]'
>>> x = ast.literal_eval(x)
>>> x
['A', 'B', 'C', ' D']
>>> x = [n.strip() for n in x]
>>> x
['A', 'B', 'C', 'D']

text = df_rxn['EC-NUMBER'].iloc[4]
text

type(text)

df_rxn.loc['PORPHOBILSYNTH-RXN']

text = df_rxn['EC-NUMBER'].loc['PORPHOBILSYNTH-RXN']

text = df_rxn['EC-NUMBER'].loc['RXN-9516']
split = text[2:-2].split("', '")
split

type(split)

split[0]

In [146]:
# count how many reactions doesn't have EC-NUMBER
counter_1 = 0
counter_m = 0
counter_n = 0

for index, row in df_rxn.iterrows():
    
    if type(row['EC-NUMBER']) != type('string'):
        #   if math.isnan(row['EC-NUMBER']):
        counter_n += 1
    else:
        data = ast.literal_eval(row['EC-NUMBER'])
        
        if len(data) == 1:
            counter_1 += 1
        elif len(data) > 1:
            counter_m += 1
        else:
            pass
print('Out of total', df_rxn.shape[0], 'row of df_rxn')
print('The data with only one, multiple, and no EC-Number are', counter_1, counter_m, 'and', counter_n, 'respectively')

Out of total 16834 row of df_rxn
The data with only one, multiple, and no EC-Number are 13251 200 and 3383 respectively


# Rearrange them into master dataframe format
Here, we have to turn all df_rxn into that of master dataframe by re-indexing with EC-Number as an index

In [44]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


df_rxn['EC-NUMBER']['RXN-9204'][0]

df_rxn.iloc[0].index

# start running here

In [53]:
EC = []
rxn = []

for index, row in df_rxn.iterrows():
    
    if len(row['EC-NUMBER']) > 1:
        for i in range(len(row['EC-NUMBER'])):
            EC.append(row['EC-NUMBER'][i])
            rxn.append(index)
    elif len(row['EC-NUMBER']) == 1:
        EC.append(row['EC-NUMBER'][0])
        rxn.append(index)
    else:
        EC.append('No_Data')
        rxn.append(index)

In [54]:
EC

['No_Data',
 'No_Data',
 'EC-4.2.1.24',
 'EC-2.5.1.74',
 'EC-2.3.1.179',
 'EC-2.3.1.41',
 'No_Data',
 'EC-2.4.1.1',
 'EC-4.2.1.59',
 'No_Data',
 'EC-6.2.1.31',
 'No_Data',
 'EC-4.1.2.51',
 'EC-2.4.1',
 'EC-2.1.1.253',
 'EC-1.1.1.M10',
 'EC-1.14.14.14',
 'EC-2.3.1.184',
 'EC-3.6.1.42',
 'EC-1.10.3.16',
 'EC-6.3.2',
 'EC-5.5.1.7',
 'EC-2.4.1',
 'EC-1.8.1.4',
 'EC-4.2.1.134',
 'No_Data',
 'EC-2.3.1.179',
 'EC-2.3.1.41',
 'EC-2.7.1.145',
 'EC-1.2.1.16',
 'No_Data',
 'EC-2.3.1.140',
 'EC-5.1.3',
 'EC-3.1.1.61',
 'EC-1.5.99',
 'EC-3.4.13.19',
 'EC-1.3.8.9',
 'EC-1.14.13',
 'EC-1.14.14.43',
 'EC-3.1.2.M5',
 'EC-4.2.3.150',
 'EC-2.7.1.217',
 'No_Data',
 'No_Data',
 'EC-2.4.1.237',
 'No_Data',
 'No_Data',
 'EC-4.1.1.38',
 'EC-1.97.1.4',
 'EC-1.1.1.76',
 'EC-1.14.14',
 'EC-1.7.7.1',
 'EC-4.4.1',
 'EC-1.14.11',
 'EC-1.1.1.385',
 'EC-2.3.1.144',
 'EC-1.8.5.8',
 'EC-3.13.1',
 'EC-5.1.3.35',
 'EC-2.3.1',
 'EC-2.3.2.2',
 'EC-1.3.1',
 'EC-2.3.1.145',
 'EC-4.4.1.30',
 'EC-5.1.3.M5',
 'No_Data',
 'No_Da

In [55]:
rxn

['RXN-12314',
 'RXN-16877',
 'PORPHOBILSYNTH-RXN',
 'RXN-9204',
 'RXN-9516',
 'RXN-9516',
 'RXN-20002',
 'RXN-14284',
 'RXN-10660',
 'RXN-12798',
 '2-FUROATE--COA-LIGASE-RXN',
 'RXN-16758',
 'DHDOGALDOL-RXN',
 'RXN-9748',
 'RXN-8133',
 'RXN-17245',
 'RXN-14746',
 'RXN-10022',
 'GUANOSINE-DIPHOSPHATASE-RXN',
 'RXN-17690',
 'RXN-16693',
 'RXN-10100',
 'RXN-4731',
 'RXN-7719',
 'RXN-13303',
 'RXN-10930',
 'RXN-16625',
 'RXN-16625',
 '2.7.1.145-RXN',
 'RXN0-5293',
 'RXN-20019',
 'RXN-7633',
 'RXN-15226',
 'MCPMETEST-RXN',
 'MTDMBARK-RXN',
 '3.4.13.19-RXN',
 'RXN-16557',
 'RXN3O-12',
 'RXN-18161',
 'RXN-19864',
 'RXN-15707',
 'RXN-18594',
 'RXN-16426',
 'RXN-11374',
 'RXN-8263',
 'RXN-19809',
 'RXN-11808',
 '4.1.1.38-RXN',
 'TDCEACT-RXN',
 'SS-BUTANEDIOL-DEHYDROGENASE-RXN',
 'RXN-8707',
 'FERREDOXIN--NITRITE-REDUCTASE-RXN',
 'RXN-16193',
 'RXN-13839',
 'RXN-16285',
 'ANTHRANILATE-N-BENZOYLTRANSFERASE-RXN',
 'RXN-19036',
 'RXN-19496',
 'RXN-16712',
 'RXN-19515',
 'RXN66-336',
 'RXN-12293',
 

In [56]:
df_master = pd.DataFrame({'EC-NUMBER' : EC,
                          'UNIQUE-ID' : rxn})

In [57]:
df_master.head()

,EC-NUMBER,UNIQUE-ID
0,No_Data,RXN-12314
1,No_Data,RXN-16877
2,EC-4.2.1.24,PORPHOBILSYNTH-RXN
3,EC-2.5.1.74,RXN-9204
4,EC-2.3.1.179,RXN-9516


In [58]:
rxn_num = []
subs = []
pdts = []
gibbs = []

for index, row in df_master.iterrows():
    ID = row['UNIQUE-ID']
    rxn_num.append(df_rxn['ERXN-NUMBER'][ID])
    subs.append(df_rxn['SUBSTRATES'][ID])
    pdts.append(df_rxn['PRODUCTS'][ID])
    gibbs.append(df_rxn['GIBBS'][ID])

In [60]:
df_master['ERXN-NUMBER'] = rxn_num
df_master['SUBSTRATES'] = subs
df_master['PRODUCTS'] = pdts
df_master['GIBBS'] = gibbs
df_master.head()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,No_Data,RXN-12314,[ENZRXN-19061],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
1,No_Data,RXN-16877,[ENZRXN-24025],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
2,EC-4.2.1.24,PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
3,EC-2.5.1.74,RXN-9204,[ENZRXN-14772],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
4,EC-2.3.1.179,RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [61]:
df_master.tail()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
17109,EC-4.2.1.59,RXN-9537,"[ENZRXN0-7977, ENZRXN-21704, ENZRXN1G-803, ENZ...",[R-3-hydroxymyristoyl-ACPs],"[Tetradec-2-enoyl-ACPs, WATER]",22.705246
17110,No_Data,ARACHIDONATE-5-LIPOXYGENASE-RXN,[ENZRXN66-1526],"[OXYGEN-MOLECULE, ARACHIDONIC_ACID]",[6E8Z11Z14Z-5S-5-HYDROPEROXYCOSA-6],-69.979996
17111,EC-1.14.13,RXN-16742,[ENZRXN-23928],"[CPD-18060, NADPH, PROTON, OXYGEN-MOLECULE]","[CPD-18056, NADP, WATER]",-82.770510
17112,EC-3.4.23.50,RXN-11145,[],"[WATER, SQNYPIVQ-Cleavage-Sites]","[Mature-P17-Matrix, P24-Capsid-Proteins]",NaN
17113,EC-2.1.1.M46,RXN-19533,[ENZRXN-26293],"[S-ADENOSYLMETHIONINE, CPD-21063]","[CPD-21067, ADENOSYL-HOMO-CYS, PROTON]",12.328987


In [13]:
df_duplicate = df_master.groupby('EC-NUMBER').size().reset_index(name='count')
df_duplicate.head()

,EC-NUMBER,count
0,EC-1,11
1,EC-1.1,26
2,EC-1.1.1,172
3,EC-1.1.1.1,15
4,EC-1.1.1.10,1


In [14]:
df_duplicate.tail()

,EC-NUMBER,count
6848,|EC-7.6.2.l|,1
6849,|EC-7.6.2.n|,1
6850,|EC-7.6.2.o|,1
6851,|EC-7.6.2.p|,1
6852,|EC-7.6.2.q|,2


In [62]:
df_sorted = df_master.sort_values(by=['EC-NUMBER'])
df_sorted.head()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
2905,EC-1,RXN0-6277,"[ENZRXN-153, ENZRXN-483]","[CPD-722, Red-Thioredoxin]","[BIOTIN, Ox-Thioredoxin, WATER]",4.545235
7737,EC-1,RXN-8705,[],"[CPD-8922, OXYGEN-MOLECULE]",[CPD-8928],-50.319992
11493,EC-1,RXN-13817,[],"[CPD-8922, OXYGEN-MOLECULE, Donor-H2]","[CPD-14837, WATER, Acceptor]",NaN
16594,EC-1,R303-RXN,[ENZRXN-503],"[Nitroaromatic-Ox-Compounds, NADPH]","[Nitroaromatic-Red-Compounds, NADP]",NaN
16307,EC-1,RXN-9620,[],"[CPD-10257, OXYGEN-MOLECULE, Donor-H2]","[CPD-10258, WATER, Acceptor]",NaN


In [63]:
df_sorted.reset_index(inplace=True, drop=True)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,EC-1,RXN0-6277,"[ENZRXN-153, ENZRXN-483]","[CPD-722, Red-Thioredoxin]","[BIOTIN, Ox-Thioredoxin, WATER]",4.545235
1,EC-1,RXN-8705,[],"[CPD-8922, OXYGEN-MOLECULE]",[CPD-8928],-50.319992
2,EC-1,RXN-13817,[],"[CPD-8922, OXYGEN-MOLECULE, Donor-H2]","[CPD-14837, WATER, Acceptor]",NaN
3,EC-1,R303-RXN,[ENZRXN-503],"[Nitroaromatic-Ox-Compounds, NADPH]","[Nitroaromatic-Red-Compounds, NADP]",NaN
4,EC-1,RXN-9620,[],"[CPD-10257, OXYGEN-MOLECULE, Donor-H2]","[CPD-10258, WATER, Acceptor]",NaN


In [67]:
df_sorted.head(20)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,EC-1,RXN0-6277,"[ENZRXN-153, ENZRXN-483]","[CPD-722, Red-Thioredoxin]","[BIOTIN, Ox-Thioredoxin, WATER]",4.545235
1,EC-1,RXN-8705,[],"[CPD-8922, OXYGEN-MOLECULE]",[CPD-8928],-50.319992
2,EC-1,RXN-13817,[],"[CPD-8922, OXYGEN-MOLECULE, Donor-H2]","[CPD-14837, WATER, Acceptor]",NaN
3,EC-1,R303-RXN,[ENZRXN-503],"[Nitroaromatic-Ox-Compounds, NADPH]","[Nitroaromatic-Red-Compounds, NADP]",NaN
4,EC-1,RXN-9620,[],"[CPD-10257, OXYGEN-MOLECULE, Donor-H2]","[CPD-10258, WATER, Acceptor]",NaN
5,EC-1,RXN-9169,[],"[OXOPENTENOATE, Acceptor]","[CPD-9734, Donor-H2]",NaN
6,EC-1,RXN-13553,"[ENZRXN-20651, ENZRXN-20646]","[CPD-14537, OXYGEN-MOLECULE, PROTON, Donor-H2]","[CPD-14536, WATER, Acceptor]",NaN
7,EC-1,RXN-8704,[],"[CPD-8927, OXYGEN-MOLECULE]","[CPD-8928, WATER]",-121.353966
8,EC-1,RXN-12254,[],"[SALICYLALDEHYDE, Donor-H2]","[CPD-173, Acceptor]",NaN
9,EC-1,RXN-8738,[],"[CPD-8961, OXYGEN-MOLECULE, Donor-H2]","[CPD-8962, WATER, Acceptor]",NaN


In [116]:
df_sorted['EC-NUMBER'][0]

'EC-1'

This cell might be wrong and lead to duplication of that EC-1

In [77]:
EC_a = 'EC-1'

EC = []
ID = []
erxn = []
subs = []
pdts = []
gibbs = []
counter = 0

ID_temp = []
erxn_temp = []
subs_temp = []
pdts_temp = []
gibbs_temp = []

for index, row in df_sorted.iterrows():
    
    if row['EC-NUMBER'] == EC_a:
        ID_temp.append(row['UNIQUE-ID'])
        erxn_temp.append(row['ERXN-NUMBER'])
        subs_temp.append(row['SUBSTRATES'])
        pdts_temp.append(row['PRODUCTS'])
        gibbs_temp.append(row['GIBBS'])
        counter += 1
        
    elif counter == 0:
        ID.append(row['UNIQUE-ID'])
        erxn.append(row['ERXN-NUMBER'])
        subs.append(row['SUBSTRATES'])
        pdts.append(row['PRODUCTS'])
        gibbs.append(row['GIBBS'])
        
        EC.append(EC_a)
        EC_a = row['EC-NUMBER']
    else:
        ID.append(ID_temp)
        erxn.append(erxn_temp)
        subs.append(subs_temp)
        pdts.append(pdts_temp)
        gibbs.append(gibbs_temp)
        
        ID_temp = []
        erxn_temp = []
        subs_temp = []
        pdts_temp = []
        gibbs_temp = []

        EC.append(EC_a)
        counter = 0
        EC_a = row['EC-NUMBER']


In [78]:
df_sorted_master = pd.DataFrame({'EC-NUMBER' : EC,
                                'UNIQUE-ID' : ID,
                                'ERXN-NUMBER' : erxn,
                                'SUBSTRATES' : subs,
                                'PRODUCTS' : pdts,
                                'GIBBS' : gibbs})

In [79]:
df_sorted_master.head(10)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,EC-1,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
1,EC-1.1,"[RXN-18132, RXN1G-1529, RXN-776, RXN-9318, RXN...","[[], [ENZRXN-15826], [], [], [], [ENZRXN-15825...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,EC-1.1.1,"[KETOGAL-DEHYDROXY-RXN, RXN-13774, RXN-17376, ...","[[], [ENZRXN-20967], [], [], [ENZRXN1G-743], [...","[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, nan, 2.54180..."
3,EC-1.1.1.1,"[RXN-10781, RXN-7694, RXN-12448, RXN-5424, RXN...","[[ENZRXN-16820], [ENZRXN3O-4113, ENZRXN3O-4112...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
4,EC-1.1.1.10,RXN-9633,"[ENZRXN1G-246, ENZRXN-15237]","[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
5,EC-1.1.1.100,"[RXN-10655, RXN1G-157, RXN-16626, RXN-9540, RX...","[[ENZRXN-16657], [ENZRXN3O-10712, ENZRXN1G-179...","[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
6,EC-1.1.1.101,"[ACYLGLYCERONE-PHOSPHATE-REDUCTASE-RXN, RXN-15...","[[ENZRXN-24829, ENZRXN3O-11401], [ENZRXN-22321...","[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
7,EC-1.1.1.102,"[3-DEHYDROSPHINGANINE-REDUCTASE-RXN, RXN-12642]","[[ENZRXN66-5214, ENZRXN-19565, ENZRXN-19561, E...","[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
8,EC-1.1.1.103,[RXN-14249],[[]],"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]
9,EC-1.1.1.104,RETINOL-DEHYDROGENASE-RXN,"[ENZRXN-19467, ENZRXN-19433]","[CPD-13524, NAD]","[RETINAL, NADH, PROTON]",0.991837


In [80]:
df_sorted_master.tail(10)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
6842,|EC-7.5.2.i|,TRANS-RXN-333,[],"[D-galactofuransoe, ATP, WATER]","[D-galactofuransoe, ADP, Pi, PROTON]",-7.59888
6843,|EC-7.5.2.j|,ABC-33-RXN,[ABC-33-ENZRXN],"[ATP, WATER, D-Xylose]","[ADP, Pi, PROTON, D-Xylose]",-7.59882
6844,|EC-7.5.2.k|,TRANS-RXN-334,[],"[D-Galactose, ATP, WATER]","[D-Galactose, ADP, Pi, PROTON]",-7.59888
6845,|EC-7.5.2.l|,ABC-2-RXN,[ABC-2-ENZRXN],"[WATER, ATP, L-ARABINOSE]","[Pi, ADP, PROTON, L-ARABINOSE]",-7.59882
6846,|EC-7.5.2.m|,ABC-33-RXN,[ABC-33-ENZRXN],"[ATP, WATER, D-Xylose]","[ADP, Pi, PROTON, D-Xylose]",-7.59882
6847,|EC-7.5.2.n|,[ABC-2-RXN],[[ABC-2-ENZRXN]],"[[WATER, ATP, L-ARABINOSE]]","[[Pi, ADP, PROTON, L-ARABINOSE]]",[-7.598816]
6848,|EC-7.6.2.l|,3.6.3.38-RXN,[],"[CAPSULAR-POLYSACCHARIDES, WATER, ATP]","[CAPSULAR-POLYSACCHARIDES, Pi, ADP]",NaN
6849,|EC-7.6.2.n|,ABC-56-RXN,[ENZRXN0-6365],"[ATP, Aliphatic-Sulfonates, WATER]","[Aliphatic-Sulfonates, ADP, Pi, PROTON]",-7.59888
6850,|EC-7.6.2.o|,ABC-32-RXN,[ABC-32-ENZRXN],"[ATP, THIAMINE, WATER]","[ADP, Pi, THIAMINE, PROTON]",-7.59888
6851,|EC-7.6.2.p|,ABC-25-RXN,"[ABC-25-ENZRXN, ABC-24A-ENZRXN]","[ATP, PUTRESCINE, WATER]","[ADP, Pi, PUTRESCINE, PROTON]",-7.59894


In [81]:
df_sorted_master.set_index(keys=['EC-NUMBER'], inplace=True)

In [20]:
df_sorted_master.index[4].count('.')

3

In [82]:
df_sorted_master

,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,,,
EC-1,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1,"[RXN-18132, RXN1G-1529, RXN-776, RXN-9318, RXN...","[[], [ENZRXN-15826], [], [], [], [ENZRXN-15825...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
EC-1.1.1,"[KETOGAL-DEHYDROXY-RXN, RXN-13774, RXN-17376, ...","[[], [ENZRXN-20967], [], [], [ENZRXN1G-743], [...","[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, nan, 2.54180..."
EC-1.1.1.1,"[RXN-10781, RXN-7694, RXN-12448, RXN-5424, RXN...","[[ENZRXN-16820], [ENZRXN3O-4113, ENZRXN3O-4112...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
EC-1.1.1.10,RXN-9633,"[ENZRXN1G-246, ENZRXN-15237]","[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
EC-1.1.1.100,"[RXN-10655, RXN1G-157, RXN-16626, RXN-9540, RX...","[[ENZRXN-16657], [ENZRXN3O-10712, ENZRXN1G-179...","[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
EC-1.1.1.101,"[ACYLGLYCERONE-PHOSPHATE-REDUCTASE-RXN, RXN-15...","[[ENZRXN-24829, ENZRXN3O-11401], [ENZRXN-22321...","[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
EC-1.1.1.102,"[3-DEHYDROSPHINGANINE-REDUCTASE-RXN, RXN-12642]","[[ENZRXN66-5214, ENZRXN-19565, ENZRXN-19561, E...","[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
EC-1.1.1.103,[RXN-14249],[[]],"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]


In [76]:
df_sorted_master.index

Index(['EC-1.1.1.1', 'EC-1.1.1.10', 'EC-1.1.1.100', 'EC-1.1.1.101',
       'EC-1.1.1.102', 'EC-1.1.1.103', 'EC-1.1.1.104', 'EC-1.1.1.105',
       'EC-1.1.1.106', 'EC-1.1.1.107',
       ...
       '|EC-7.5.2.i|', '|EC-7.5.2.j|', '|EC-7.5.2.k|', '|EC-7.5.2.l|',
       '|EC-7.5.2.m|', '|EC-7.5.2.n|', '|EC-7.6.2.l|', '|EC-7.6.2.n|',
       '|EC-7.6.2.o|', '|EC-7.6.2.p|'],
      dtype='object', name='EC-NUMBER', length=6682)

In [93]:
drop = []
for index, row in df_sorted_master.iterrows():
    if index.count('.') < 'EC-1.1.1.1'.count('.'):
        #print(index)
        drop.append(index)

In [94]:
drop[:10]

['EC-1',
 'EC-1.1',
 'EC-1.1.1',
 'EC-1.1.3',
 'EC-1.1.5',
 'EC-1.1.99',
 'EC-1.10.3',
 'EC-1.11.1',
 'EC-1.11.2',
 'EC-1.13.11']

In [95]:
df_sorted_master_drop = df_sorted_master
for item in drop:
    df_sorted_master_drop = df_sorted_master_drop.drop(item)

In [96]:
for index, row in df_sorted_master_drop.iterrows():
    if index.count('.') < 'EC-1.1.1.1'.count('.'):
        print(index)

In [99]:
df_sorted_master_drop.drop(['UNIQUE-ID', 'ERXN-NUMBER'], axis=1)

,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,
EC-1.1.1.1,"[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
EC-1.1.1.10,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
EC-1.1.1.100,"[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
EC-1.1.1.101,"[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
EC-1.1.1.102,"[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
EC-1.1.1.103,"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]
EC-1.1.1.104,"[CPD-13524, NAD]","[RETINAL, NADH, PROTON]",0.991837
EC-1.1.1.105,"[[CRPB-all-trans-Retinol, NAD]]","[[CRPB-all-trans-Retinal, NADH, PROTON]]",[3.2340088]
EC-1.1.1.106,"[PYRIDOXAL, NAD]","[CPD-375, NADH, PROTON]",-4.83823


This approach can be done by changing the substrates and products in df_rxn into PubChemID first ('0' for NaN)

In [100]:
# the file is too big!!! up to 1.7 G --- should reduce the size first
df_sorted_master.to_csv('df_master_1st.csv')

# changing to PubChemID still give 1.3 G left

# here is the first version of master dataframe
However, it is not perfect
- CPD-ID is not converted to PubChemID or InChI yet
- cofactor is not cleaned
- reversibility is not applied
- Anyway, should be in similar shape to that of previous master but contains single reactions

In [25]:
df_prom = df_sorted_master
for index, row in df_sorted_master.iterrows():
    if type(row['UNIQUE-ID']) == type('string'):
        df_prom = df_prom.drop(index)

In [75]:
df_prom.index

Index(['EC-1.1.1.1', 'EC-1.1.1.100', 'EC-1.1.1.101', 'EC-1.1.1.102',
       'EC-1.1.1.103', 'EC-1.1.1.105', 'EC-1.1.1.110', 'EC-1.1.1.130',
       'EC-1.1.1.14', 'EC-1.1.1.144',
       ...
       '|EC-6.2.1.ah|', '|EC-6.2.1.v|', '|EC-6.2.1.w|', '|EC-6.2.1.y|',
       '|EC-6.2.1.z|', '|EC-6.3.2.z|', '|EC-7.2.2.i|', '|EC-7.4.2.j|',
       '|EC-7.4.2.k|', '|EC-7.5.2.n|'],
      dtype='object', name='EC-NUMBER', length=1551)

In [77]:
# this is still too big??? like 1.8 G, how to reduce it? ---> turn into PubChemID?
df_prom.to_csv('df_prom.csv')

In [26]:
df_prom.head()

,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,,,
EC-1.1.1.1,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.100,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.101,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.102,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.103,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."


In [27]:
df_prom = df_prom.drop(['UNIQUE-ID', 'ERXN-NUMBER'], axis = 1)

In [28]:
df_prom.head()

,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,
EC-1.1.1.1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.100,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.101,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.102,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.103,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."


In [34]:
# from metamoles.py
def parse_reversible_reactions(reaction_list: list):
    """
    parse_reversible_reactions() queries the KEGG database with the input
        reaction list, and parses the results for all reactions that have been
        annotated with "<=>" in the reaction equation, which suggests that the
        catalyzed reaction is reversible

    Args:
        reaction_list (list): contains KEGG reaction IDs (e.g. 'R00709')

    Returns:
        list: contains KEGG IDs of reversible reactions
    """

    reversible_reaction = []
    for reaction in reaction_list:
        reaction_file = REST.kegg_get(reaction).read()
        for i in reaction_file.rstrip().split("\n"):
            if i.startswith("EQUATION") and "<=>" in i:
                reversible_reaction.append(reaction)
    return reversible_reaction

def combine_substrates_products(df: pd.DataFrame):
    """
    combine_substrates_products() is for use with a collection of enzymes
        in which it is understood that they are capable of catalyzing both the
        forward and reverser reactions. In this case, both the substrates and
        the products should be considered as bioreachable products.
        This function parses the list of substrates and products from their
        respective fields in the input dataframe, and returns a new dataframe
        with the combined substrates and products in a column labeled 'product'

    WARNING: combine_substrates_products() should not be run multiple times on
        the same dataframe becuase it will will append duplicate substrates

    Args:
        df (pandas.DataFrame): must contain fields
            ['entry', 'substrate', 'product']

    Returns:
        pandas.DataFrame: contains only fields ['entry', 'product']
    """

    rowindex = np.arange(0,len(df))
    df_with_ordered_index = df.set_index(rowindex)

    newdf = df_with_ordered_index
    # should this be a .copy()?

    for index,row in df_with_ordered_index.iterrows():
        productlist = row['product']
        substratelist = row['substrate']
        newdf.iloc[index,2] = productlist + substratelist

    return newdf[["entry","product"]]


def explode_dataframe(dataframe: pd.DataFrame, explosion_function,
                        explosion_target_field: str, fields_to_include: list):
    """
    explode_dataframe() applies the input explosion_function to the target
        field in each row of a dataframe. Each item in the output of the
        explosion_function is an anchor for a new row in the new dataframe. All
        of the supplied fields_to_include are added to the explosion item,
        and appended to the new dataframe row.

    Args:
        dataframe (pandas.DataFrame): input dataset
        explosion_function (function): function to be applied to target
            column in dataframe
        explosion_target_field (str): name of field in dataframe to which the
            explosion funciton will be applied
        fields_to_include (list): a list of strings that denote the columns of
            the input dataframe to be included in the output

    Returns:
        pandas.DataFrame: new exploded dataframe
    """
    new_rows = []
    for _, row in dataframe.iterrows():
        explosion_list = explosion_function(row[explosion_target_field])
        for item in explosion_list:
            row_data = [row[field] for field in fields_to_include]
            row_data.append(item)
            new_rows.append(row_data)

    fields_to_include.append(explosion_target_field)
    new_df = pd.DataFrame(new_rows, columns=fields_to_include)

    return new_df


def remove_cofactors(master_df: pd.DataFrame, master_cpd_field: str,
                     cofactor_df: pd.DataFrame, cofactor_field: str,
                     drop_na=True):
    """
    remove_cofactors() should be used to clean the dataset of cofactors. These
        will be included in the KEGG records as substrates and products, but
        are not actually products in the reaction

    Args:
        master_df (pandas.DataFrame): input dataset
        master_cpd_field (str): field that contains products
        cofactor_df (pandas.DataFrame): contains cofactors to be removed
        cofactor_field (str): field that contains cofactors
        drop_na (bool): default True

    Returns:
        pandas.DataFrame: cleaned data without cofactor entries
    """
    cofactor_list = parse_compound_ids(cofactor_df[cofactor_field])
    bool_mask = [False if cpd in cofactor_list else True for cpd in master_df[master_cpd_field]]
    clean_df = master_df[bool_mask]
    clean_df = clean_df.drop_duplicates()

    if drop_na:
        clean_df = clean_df[clean_df[master_cpd_field] != 'NA']
    else:
        pass

    return clean_df

def create_negative_matches(dataframe: pd.DataFrame,
                            enzyme_field: str, compound_field: str):
    """
    create_negative_matches() returns two dataframes.
        One dataframe is positive data that contains all the enzyme-compound
        pairs that exist in the input dataset.
        The second data frame is negative data made from matching all
        enzyme-compound pairs that do not exist in the dataset.

    Args:
        dataframe (pandas.DataFrame): input dataset
        enzyme_field (str): column in dataframe that contains enzyme ids
        compound_field (str): column in dataframe that contains compound ids

    Returns:
        pandas.DataFrame: positive data
            (contains fields ['enzyme', 'product', 'reacts'])
        pandas.DataFrame: negative data
            (contains fields ['enzyme', 'product', 'reacts'])
    """
    unique_enzymes = set(dataframe[enzyme_field].unique())
    # set of all unique enzymes in provided dataframes
    unique_cpds = set(dataframe[compound_field].unique())
    # set of all unique compounds in provided dataframe

    positive_data = []
    negative_data = []
    # initialize empty lists

    for enzyme in unique_enzymes:
    # iterate through unique enzyme set
        working_prods = set(dataframe[dataframe[enzyme_field] == enzyme][compound_field].unique())
        # unique set of all products reported to reaction with this enzyme in provided dataset
        non_working_prods = (unique_cpds - working_prods)
        # set math of all remaining products in the dataset minus those reported to react

        reactions = [{'reacts':1.0, 'enzyme':enzyme, 'product':product} for product in working_prods]
        # create new entry for each positive reaction
        non_reactions = [{'reacts':0.0, 'enzyme':enzyme, 'product':product} for product in non_working_prods]
        # create new entry for each negative reaction

        positive_data.extend(reactions)
        # add positive reactions to master list
        negative_data.extend(non_reactions)
        # add negative reactions to master list

    positive_df = pd.DataFrame(positive_data)
    negative_df = pd.DataFrame(negative_data)

    return positive_df, negative_df



In [32]:
df_prom['SUBS-REV'] = df_prom['PRODUCTS']
df_prom['PDTS-REV'] = df_prom['SUBSTRATES']
df_prom['GIBBS-REV'] = df_prom['GIBBS'] # should be reverse but will fix this later

In [38]:
df_prom.reset_index(inplace=True)

In [39]:
df_prom

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,SUBS-REV,PDTS-REV,GIBBS-REV
0,EC-1.1.1.1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
1,EC-1.1.1.100,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
2,EC-1.1.1.101,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
3,EC-1.1.1.102,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
4,EC-1.1.1.103,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
5,EC-1.1.1.105,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
6,EC-1.1.1.110,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
7,EC-1.1.1.130,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
8,EC-1.1.1.14,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
9,EC-1.1.1.144,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."


In [40]:
create_negative_matches(df_prom, 'EC-NUMBER', 'SUBSTRATES')

TypeError: unhashable type: 'list'

In [85]:
for index, row in df_prom.iterrows():
    for item in row['SUBSTRATES']:
        for sub in item:
            sub = get_pubchem(sub)

In [88]:
for index, row in df_prom.iterrows():
    subs = []
    for item in row['SUBSTRATES']:
        subs_2 = []
        for sub in item:
            subs_2.append(get_pubchem(sub))
        subs.append(subs_2)
    df_prom['SUBSTRATES'][index] = subs

KeyboardInterrupt: 

In [86]:
df_prom.head()

,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,
EC-1.1.1.1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.100,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.101,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.102,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."
EC-1.1.1.103,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, nan, nan, nan, nan, na..."


#### Still, there is a problem
For some `EC-NUMBER` value, they are in `|_|` bracket with weird annotation with character instead of number. To solve this, we can modify some string value but if there is no repeating `EC-NUMBER` with and without bracket, we can just change the `EC-NUMBER` column without renewing the dataframe.

## So, we have a sortof Master Dataframe v1 right now

### What is the next step?
- Alter CPD-ID into PubChemID or InChI ---> go for PubChemID first
    - Done!
- Make reversible reaction
- Make negative dataset

In [5]:
def get_pubchem(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    if ID in df_cpd['PubChemID']:
        pubchem = df_cpd['PubChemID'][ID]
    else:
        pubchem = '0'
        
    return pubchem

In [6]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [16]:
'Nitroaromatic-Ox-Compounds' in df_cpd['PubChemID']

False

In [14]:
'BIOTIN' in df_cpd['PubChemID']

True

In [30]:
if 'BIOTIN' in df_cpd['PubChemID']:
    print('Yes')

Yes


In [39]:
get_pubchem('MALONYL-ACP')

'MALONYL-ACP'

In [7]:
# Start from df_rxn and rerun the master dataframe again
subs_id = []
pdts_id = []

for index, row in df_rxn.iterrows():
    
    subs = []
    for item in row['SUBSTRATES']:
        subs.append(get_pubchem(item))
    subs_id.append(subs)
    
    pdts = []
    for item in row['PRODUCTS']:
        pdts.append(get_pubchem(item))
    pdts_id.append(pdts)

In [8]:
df_rxn['SUBSTRATES'] = subs_id
df_rxn['PRODUCTS'] = pdts_id
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[15983961, 0, 977]","[52940136, 0, 962, 1038]",-108.408860
RXN-16877,[ENZRXN-24025],[],[0],[0],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[7048523],"[1038, 962, 6921588]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[25244603, 54706667, 1038]","[45479709, 1023, 280]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[0, 0, 1038]","[0, 280, 0]",-6.862854


In [45]:
counter_s_y = 0
counter_s_n = 0

counter_p_y = 0
counter_p_n = 0

for index, row in df_rxn.iterrows():
    
    for item in row['SUBSTRATES']:
        if item in df_cpd['PubChemID']:
            counter_s_y += 1
        else:
            counter_s_n += 1
    
    for item in row['PRODUCTS']:
        if item in df_cpd['PubChemID']:
            counter_p_y += 1
        else:
            counter_p_n += 1

print('Counting substrates True is', counter_s_y, 'False is', counter_s_n)
print('Counting products True is', counter_p_y, 'False is', counter_p_n)

Counting substrates True is 29253 False is 8287
Counting products True is 34480 False is 7752


In [49]:
percent_s = counter_s_y/(counter_s_y + counter_s_n)
percent_p = counter_p_y/(counter_p_y + counter_p_n)

print('Percents of available substrate and product are', percent_s*100, 'and', percent_p*100)

Percents of available substrate and product are 77.92488012786362 and 81.64425080507672


In [139]:
get_pubchem('Nitroaromatic-Ox-Compounds')

KeyError: 'Nitroaromatic-Ox-Compounds'

### Problems found
Where there is no data available for a certain UNIQUE-ID, the function goes wrong!!!
- Those are macromolecules, general group of compounds, etc.

Already solve by check availability by `in` function